# **Welcom to MMTracking**

In this tutorial, you will learn to:
+ install MMTracking.
+ perform inference with pretrained weights in MMTracking.
+ train a new MOT model with a toy dataset.
Let's start!

## **Install MMTracking**

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install MMCV
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install MMDetection
!pip install mmdet

# clone the MMTracking repository
!git clone https://github.com/open-mmlab/mmtracking.git
%cd mmtracking

# install MMTracking and its dependencies
!pip install -r requirements/build.txt
!pip install -e .
# used to MOT evaluation
!pip install git+https://github.com/JonathonLuiten/TrackEval.git

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 45.7 MB 20 kB/s 
     |████████████████████████████████| 190 kB 32.1 MB/s 
     |████████████████████████████████| 1.4 MB 26.1 MB/s 
Cloning into 'mmtracking'...
remote: Enumerating objects: 4182, done.
remote: Counting objects: 100% (4182/4182), done.
remote: Compressing objects: 100% (1631/1631), done.
remote: Total 4182 (delta 2499), reused 3979 (delta 2433), pack-reused 0
Receiving objects: 100% (4182/4182), 1.67 MiB | 26.35 MiB/s, done.
Resolving deltas: 100% (2499/2499), done.
/content/mmtracking
Obtaining file:///content/mmtracking
     |████████████████████████████████| 1.5 MB 52.6 MB/s 
     |████████████████████████████████| 548 kB 44.2 MB/s 
     |████████████████████████████████| 161 kB 78.3 MB/s 
  Created wheel for attributee: filename=attributee-0.1.5-py3-none-any.whl size=12076 sha256=b32a3b03b3e8a159229816c3bc0bbb33c404f75ad50210db573a79a2a1f8

In [ ]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla T4',
 'MMCV': '1.4.8',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Build cuda_11.1.TC455_06.29190527_0',
 'OpenCV': '4.1.2',
 'PyTorch': '1.10.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check MMTracking installation
import mmtrack
print(mmtrack.__version__)

1.10.0+cu111 True
11.1
GCC 7.3
2.23.0
0.12.0


## **Perform inference**

In [ ]:
# unset the proxy for downloading the pretrained models (optional)
!unset https_proxy
!unset http_proxy

# download checkpoints
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth -P ./checkpoints
!wget -c https://download.openmmlab.com/mmtracking/sot/siamese_rpn/siamese_rpn_r50_1x_lasot/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth -P ./checkpoints
!wget -c https://download.openmmlab.com/mmtracking/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth -P ./checkpoints

--2022-04-19 03:05:57--  https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.74.197.77
Connecting to download.openmmlab.com (download.openmmlab.com)|47.74.197.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282801031 (270M) [application/octet-stream]
Saving to: ‘./checkpoints/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth’

selsa_faster_rcnn_r 100%[===================>] 269.70M  7.38MB/s    in 37s     

2022-04-19 03:06:35 (7.20 MB/s) - ‘./checkpoints/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth’ saved [282801031/282801031]

--2022-04-19 03:06:35--  https://download.openmmlab.com/mmtracking/sot/siamese_rpn/siamese_rpn_r50_1x_lasot/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth
Resolving download.openmmlab.com (download.openmmlab.co

In [ ]:
# run mot demo
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model
mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py'
input_video = './demo/demo.mp4'
imgs = mmcv.VideoReader(input_video)
# build the model from a config file
mot_model = init_model(mot_config, device='cuda:0')
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
# test and show/save the images
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

output = './demo/mot.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-19 03:07:51,190 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth'}
2022-04-19 03:07:51,191 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth
2022-04-19 03:07:51,195 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth
Downloading: "https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth" to /root/.cache/torch/hub/checkpoints/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth


  0%|          | 0.00/158M [00:00<?, ?B/s]

2022-04-19 03:08:14,890 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth'}
2022-04-19 03:08:14,892 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth
2022-04-19 03:08:14,894 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth
Downloading: "https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth" to /root/.cache/torch/hub/checkpoints/tracktor_reid_r50_iter25245-a452f51f.pth


  0%|          | 0.00/98.4M [00:00<?, ?B/s]

2022-04-19 03:08:25,432 - mmcv - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.running_var, head.classifier.weight, head.classifier.bias



[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 3.5 task/s, elapsed: 2s, ETA:     0s
 making the output video at ./demo/mot.mp4 with a FPS of 3.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 23.9 task/s, elapsed: 0s, ETA:     0s


In [ ]:
# run vis demo
from mmtrack.apis import inference_mot
vis_config = './configs/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019.py'
vis_checkpoint = './checkpoints/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth'
# build the model from a config file and a checkpoint file
vis_model = init_model(vis_config, vis_checkpoint, device='cuda:0')
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
for i, img in enumerate(imgs):
    result = inference_mot(vis_model, img, frame_id=i)
    vis_model.show_result(
            img,
            result,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()
output = './demo/vis.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-19 03:08:47,571 - mmtrack - INFO - initialize MaskRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'}
2022-04-19 03:08:47,573 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
2022-04-19 03:08:47,576 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
Downloading: "https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth" to /root/.cache/torch/hub/checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

2022-04-19 03:09:08,040 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([41, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([41]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([160, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([160]).
size mismatch for roi_head.mask_head.conv_logits.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([40, 256, 1, 1]).
size mismatch for roi_head.mask_head.conv_logit

load checkpoint from local path: ./checkpoints/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 3.6 task/s, elapsed: 2s, ETA:     0s
 making the output video at ./demo/vis.mp4 with a FPS of 3.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 25.0 task/s, elapsed: 0s, ETA:     0s


In [ ]:
# run vid demo
from mmtrack.apis import inference_vid
vid_config = './configs/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid.py'
vid_checkpoint = './checkpoints/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth'
# build the model from a config file and a checkpoint file
vid_model = init_model(vid_config, vid_checkpoint, device='cuda:0')
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
for i, img in enumerate(imgs):
    result = inference_vid(vid_model, img, frame_id=i)
    vid_model.show_result(
            img,
            result,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()
output = './demo/vid.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-19 03:09:24,641 - mmtrack - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet101'}
2022-04-19 03:09:24,643 - mmcv - INFO - load model from: torchvision://resnet101
2022-04-19 03:09:24,644 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet101
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

2022-04-19 03:09:30,061 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-04-19 03:09:30,120 - mmtrack - INFO - initialize ChannelMapper with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-19 03:09:30,188 - mmtrack - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-19 03:09:30,207 - mmtrack - INFO - initialize SelsaBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]


load checkpoint from local path: ./checkpoints/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: detector.backbone.layer3.6.conv1.weight, detector.backbone.layer3.6.bn1.weight, detector.backbone.layer3.6.bn1.bias, detector.backbone.layer3.6.bn1.running_mean, detector.backbone.layer3.6.bn1.running_var, detector.backbone.layer3.6.conv2.weight, detector.backbone.layer3.6.bn2.weight, detector.backbone.layer3.6.bn2.bias, detector.backbone.layer3.6.bn2.running_mean, detector.backbone.layer3.6.bn2.running_var, detector.backbone.layer3.6.conv3.weight, detector.backbone.layer3.6.bn3.weight, detector.backbone.layer3.6.bn3.bias, detector.backbone.layer3.6.bn3.running_mean, detector.backbone.layer3.6.bn3.running_var, detector.backbone.layer3.7.conv1.weight, detector.backbone.layer3.7.bn1.weight, detector.backbone.layer3.7.bn1.bias, detector.backbone.layer3.7.bn1.running_mean, detector.backb

/content/mmtracking/mmtrack/datasets/pipelines/formatting.py:138: UserWarning: The 'ConcatVideoReferences' class will be deprecated in the future, please use 'ConcatSameTypeFrames' instead
  "The 'ConcatVideoReferences' class will be deprecated in the "


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 2.1 task/s, elapsed: 4s, ETA:     0s
 making the output video at ./demo/vid.mp4 with a FPS of 3.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 24.6 task/s, elapsed: 0s, ETA:     0s


In [ ]:
# run sot demo
from mmtrack.apis import inference_sot
sot_config = './configs/sot/siamese_rpn/siamese_rpn_r50_20e_lasot.py'
sot_checkpoint = './checkpoints/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth'
# build the model from a config file and a checkpoint file
sot_model = init_model(sot_config, sot_checkpoint, device='cuda:0')
init_bbox = [371, 411, 450, 646]
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
for i, img in enumerate(imgs):
    result = inference_sot(sot_model, img, init_bbox, frame_id=i)
    sot_model.show_result(
            img,
            result,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()
output = './demo/sot.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-19 03:11:58,458 - mmtrack - INFO - initialize SOTResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/pretrained_weights/sot_resnet50.model'}
2022-04-19 03:11:58,459 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/pretrained_weights/sot_resnet50.model
2022-04-19 03:11:58,462 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/pretrained_weights/sot_resnet50.model
Downloading: "https://download.openmmlab.com/mmtracking/pretrained_weights/sot_resnet50.model" to /root/.cache/torch/hub/checkpoints/sot_resnet50.model


  0%|          | 0.00/174M [00:00<?, ?B/s]

load checkpoint from local path: ./checkpoints/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 8.9 task/s, elapsed: 1s, ETA:     0s
 making the output video at ./demo/sot.mp4 with a FPS of 3.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 24.0 task/s, elapsed: 0s, ETA:     0s


## **Train a MOT model with a toy dataset**

### **Prepare dataset**

In [ ]:
!mkdir data
!wget https://download.openmmlab.com/mmtracking/data/MOT17_tiny.zip -P ./data
!unzip -q ./data/MOT17_tiny.zip -d ./data

--2022-04-19 03:16:45--  https://download.openmmlab.com/mmtracking/data/MOT17_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.74.197.77
Connecting to download.openmmlab.com (download.openmmlab.com)|47.74.197.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344566302 (329M) [application/zip]
Saving to: ‘./data/MOT17_tiny.zip’

MOT17_tiny.zip      100%[===================>] 328.60M  11.3MB/s    in 29s     

2022-04-19 03:17:16 (11.2 MB/s) - ‘./data/MOT17_tiny.zip’ saved [344566302/344566302]



In [ ]:
# convert the dataset to coco format
!python ./tools/convert_datasets/mot/mot2coco.py -i ./data/MOT17_tiny/ -o ./data/MOT17_tiny/annotations --split-train --convert-det
# crop pedestrian patches from the original dataset for training reid model. It may take a few minutes.
!rm -rf ./data/MOT17_tiny/reid
!python ./tools/convert_datasets/mot/mot2reid.py -i ./data/MOT17_tiny/ -o ./data/MOT17_tiny/reid --val-split 0.9 --vis-threshold 0.8

Converting train set to COCO format
100% 2/2 [00:00<00:00,  2.71it/s]
train has 145 instances.
Done! Saved as ./data/MOT17_tiny/annotations/train_cocoformat.json and ./data/MOT17_tiny/annotations/train_detections.pkl
Converting test set to COCO format
0it [00:00, ?it/s]
test has 0 instances.
Done! Saved as ./data/MOT17_tiny/annotations/test_cocoformat.json and ./data/MOT17_tiny/annotations/test_detections.pkl
Converting half-train set to COCO format
100% 2/2 [00:01<00:00,  1.05it/s]
half-train has 104 instances.
Done! Saved as ./data/MOT17_tiny/annotations/half-train_cocoformat.json and ./data/MOT17_tiny/annotations/half-train_detections.pkl
Converting half-val set to COCO format
100% 2/2 [00:01<00:00,  1.07it/s]
half-val has 122 instances.
Done! Saved as ./data/MOT17_tiny/annotations/half-val_cocoformat.json and ./data/MOT17_tiny/annotations/half-val_detections.pkl
100% 2/2 [08:44<00:00, 262.26s/it]


### **Train a detector for MOT**

In [ ]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_4e_mot17-half.py')
cfg.data_root = 'data/MOT17_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/MOT17/','data/MOT17_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')

cfg.work_dir = './tutorial_exps/detector'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

In [ ]:
import os.path as osp

from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
model = build_model(cfg.model.detector)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg)

2022-04-19 03:28:02,182 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'}
2022-04-19 03:28:02,186 - mmcv - INFO - load model from: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-04-19 03:28:02,186 - mmcv - INFO - load checkpoint from http path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Downloading: "http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth" to /root/.cache/torch/hub/checkpoints/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8

  0%|          | 0.00/160M [00:00<?, ?B/s]

2022-04-19 03:28:21,471 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([4]).


loading annotations into memory...


/usr/local/lib/python3.7/dist-packages/mmdet/apis/train.py:135: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
2022-04-19 03:28:21,841 - mmdet - INFO - Start running, host: root@de643fd9c128, work_dir: /content/mmtracking/tutorial_exps/detector
2022-04-19 03:28:21,842 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL)

Done (t=0.24s)
creating index...
index created!


2022-04-19 03:28:47,478 - mmdet - INFO - Epoch [1][50/414]	lr: 9.902e-03, eta: 0:13:38, time: 0.510, data_time: 0.056, memory: 4146, loss_rpn_cls: 0.0897, loss_rpn_bbox: 0.1169, loss_cls: 0.4091, acc: 80.7734, loss_bbox: 0.3503, loss: 0.9660
2022-04-19 03:29:11,294 - mmdet - INFO - Epoch [1][100/414]	lr: 1.980e-02, eta: 0:12:46, time: 0.476, data_time: 0.010, memory: 4146, loss_rpn_cls: 0.0452, loss_rpn_bbox: 0.1078, loss_cls: 0.3113, acc: 86.7949, loss_bbox: 0.2217, loss: 0.6861
2022-04-19 03:29:34,630 - mmdet - INFO - Epoch [1][150/414]	lr: 2.000e-02, eta: 0:12:09, time: 0.467, data_time: 0.011, memory: 4146, loss_rpn_cls: 0.0311, loss_rpn_bbox: 0.1012, loss_cls: 0.2986, acc: 87.2930, loss_bbox: 0.2179, loss: 0.6489
2022-04-19 03:29:57,682 - mmdet - INFO - Epoch [1][200/414]	lr: 2.000e-02, eta: 0:11:36, time: 0.461, data_time: 0.009, memory: 4146, loss_rpn_cls: 0.0274, loss_rpn_bbox: 0.0823, loss_cls: 0.2828, acc: 87.9844, loss_bbox: 0.2004, loss: 0.5929
2022-04-19 03:30:21,188 - mmd

### **Train a ReID model for MOT**

In [ ]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/reid/resnet50_b32x8_MOT17.py')
cfg.data_root = 'data/MOT17_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.ann_file = 'data/MOT17_tiny/reid/meta/train_9.txt'
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/MOT17/','data/MOT17_tiny/')

cfg.data.test.data_prefix = cfg.data.test.data_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.data_prefix = cfg.data.train.data_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.data_prefix = cfg.data.val.data_prefix.replace('data/MOT17/','data/MOT17_tiny/')

# learning policy
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])
cfg.total_epochs = 2

cfg.work_dir = './tutorial_exps/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(
        type='SeqNormalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),

In [ ]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model


model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

train_model(model, datasets, cfg)

2022-04-19 03:47:18,560 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2022-04-19 03:47:18,562 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-04-19 03:47:18,564 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth" to /root/.cache/torch/hub/checkpoints/resnet50_batch256_imagenet_20200708-cfb998bf.pth


  0%|          | 0.00/97.7M [00:00<?, ?B/s]

2022-04-19 03:47:28,425 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.running_var, head.classifier.weight, head.classifier.bias

/usr/local/lib/python3.7/dist-packages/mmdet/apis/train.py:135: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  'please set `runner` in your config.', UserWarning)
2022-04-19 03:47:28,495 - mmdet - INFO - Start running, host: root@de643fd9c128, work_dir: /content/mmtracking/tutorial_exps/reid
2022-04-19 03:47:28,496 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMA

### **Test the DeepSORT model**

In [ ]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')
cfg.data_root = 'data/MOT17_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.ann_file = cfg.data.test.ann_file.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/MOT17/','data/MOT17_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/MOT17/','data/MOT17_tiny/')

cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'

cfg.work_dir = './tutorial_exps'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.test.test_mode = True
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

In [ ]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)
print(metric)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


2022-04-19 04:15:43,150 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
INFO:mmtrack:initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-04-19 04:15:43,155 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
INFO:mmcv:load model from: ./tutorial_exps/detector/epoch_4.pth
2022-04-19 04:15:43,160 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
INFO:mmcv:load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-04-19 04:15:43,482 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
INFO:mmtrack:initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-04-19 04:15:43,486 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
INFO:mmcv:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 823/823, 4.9 task/s, elapsed: 168s, ETA:     0sEvaluate CLEAR MOT results.

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /usr/local/lib/python3.7/dist-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
GT_FOLDER            : data/MOT17_tiny/train         
TRACKERS_FOLDER    

<Figure size 432x288 with 0 Axes>